In [29]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from tabulate import tabulate
from IPython.display import Markdown, display

def calculate_metrics_for_models(model_results, task="", tablefmt="grid"):
    """
    Given a list of tuples (model_name, file_path, row_color, section_break), compute and print classification metrics.

    Parameters:
    - model_results (list of tuples): Each tuple is (model_name, file_path, color, section_break).
    - task (str): Title printed above the table.
    - tablefmt (str): Style passed to tabulate (e.g., 'github', 'grid').
    """
    COLORS = {
        "green": "\033[92m",
        "red": "\033[91m",
        "yellow": "\033[93m",
        "blue": "\033[94m",
        "magenta": "\033[95m",
        "cyan": "\033[96m",
        "white": "\033[97m",
        None: ""
    }
    RESET = "\033[0m"

    table_rows = []

    for model_name, file_path, color, section_break in model_results:
        try:
            df = pd.read_json(file_path)

            if "y_true" not in df or "y_pred" not in df:
                print(f"Error: Missing 'y_true' or 'y_pred' in {file_path}")
                continue

            y_true = df["y_true"]
            y_pred = df["y_pred"]

            
            accuracy = round(accuracy_score(y_true, y_pred) * 100, 2)
            precision = round(precision_score(y_true, y_pred, average="weighted", zero_division=0) * 100, 2)
            recall = round(recall_score(y_true, y_pred, average="weighted") * 100, 2)
            f1 = round(f1_score(y_true, y_pred, average="weighted") * 100, 2)

            #for debug
            #from sklearn.metrics import classification_report
            #print(classification_report(y_true, y_pred,digits=4))

            row = [model_name, accuracy, precision, recall, f1]

            # Apply color if specified
            color_code = COLORS.get(color, color)
            if color_code:
                row = [f"{color_code}{val}{RESET}" for val in row]

            table_rows.append(row)

            # Insert a blank row if section_break is True
            if section_break:
                table_rows.append([""] * len(row))

        except Exception as e:
            print(f"Error processing {model_name} ({file_path}): {e}")

    headers = ["Model", "Accuracy (%)", "Precision (%)", "Recall (%)", "F1 Score (%)"]
    print(f"\n📊 **{task} Performance Metrics**\n")
    print(tabulate(table_rows, headers=headers, tablefmt=tablefmt))

def calculate_metrics_per_category(model_results, task="", tablefmt="grid"):
    """
    Given a list of tuples (model_name, file_path, row_color, section_break), compute and print per-category metrics.

    Parameters:
    - model_results (list of tuples): Each tuple is (model_name, file_path, color, section_break).
    - task (str): Title printed above the table.
    - tablefmt (str): Style passed to tabulate (e.g., 'github', 'grid').
    """
    COLORS = {
        "green": "\033[92m",
        "red": "\033[91m",
        "yellow": "\033[93m",
        "blue": "\033[94m",
        "magenta": "\033[95m",
        "cyan": "\033[96m",
        "white": "\033[97m",
        None: ""
    }
    RESET = "\033[0m"

    for model_name, file_path, color, section_break in model_results:
        try:
            df = pd.read_json(file_path)

            if "y_true" not in df or "y_pred" not in df:
                print(f"Error: Missing 'y_true' or 'y_pred' in {file_path}")
                continue

            y_true = df["y_true"]
            y_pred = df["y_pred"]

            report = classification_report(y_true, y_pred, digits=2, output_dict=True, zero_division=0)
            classes = [key for key in report.keys() if key not in ["accuracy", "macro avg", "weighted avg"]]

            table_rows = []
            for cls in classes:
                cls_metrics = report[cls]
                row = [
                    cls,
                    round(cls_metrics["precision"] * 100, 2),
                    round(cls_metrics["recall"] * 100, 2),
                    round(cls_metrics["f1-score"] * 100, 2),
                    int(cls_metrics["support"])
                ]
                color_code = COLORS.get(color, "")
                if color_code:
                    row = [f"{color_code}{v}{RESET}" for v in row]
                table_rows.append(row)

            headers = ["Class", "Precision (%)", "Recall (%)", "F1 Score (%)", "Support"]
            print(f"\n📊 **{task} — {model_name} Per-Category Metrics**\n")
            print(tabulate(table_rows, headers=headers, tablefmt=tablefmt))

            if section_break:
                print("\n" + "=" * 80 + "\n")

        except Exception as e:
            print(f"Error processing {model_name} ({file_path}): {e}")   

def calculate_metrics_with_category_breakdown(model_results, task="", tablefmt_summary="grid", tablefmt_category="grid"):
    """
    Computes overall metrics and per-category breakdown for each model, with automatic label descriptions,
    and adds an overall metrics row to each breakdown table. Uses Markdown display for Jupyter bold headers.
    """

    COLORS = {
        "green": "\033[92m",
        "red": "\033[91m",
        "yellow": "\033[93m",
        "blue": "\033[94m",
        "magenta": "\033[95m",
        "cyan": "\033[96m",
        "white": "\033[97m",
        None: ""
    }
    RESET = "\033[0m"

    # Label descriptions for Humanitarian task
    description_humanitarian = {
        "0": "Affected individuals",
        "1": "Rescue/volunteering/donation",
        "2": "Infrastructure & utility damage",
        "3": "Other relevant info",
        "4": "Not humanitarian"
    }

    # Label descriptions for Informative task
    description_informative = {
        "0": "Not informative",
        "1": "Informative"
    }

    # Choose correct description dict based on task name
    if "Humanitarian" in task:
        description_dict = description_humanitarian
    elif "Informative" in task:
        description_dict = description_informative
    else:
        description_dict = {}

    summary_rows = []
    per_model_breakdowns = []

    for model_name, file_path, color, section_break in model_results:
        try:
            df = pd.read_json(file_path)

            if "y_true" not in df or "y_pred" not in df:
                print(f"Error: Missing 'y_true' or 'y_pred' in {file_path}")
                continue

            y_true = df["y_true"].astype(str)
            y_pred = df["y_pred"].astype(str)

            # Overall metrics
            accuracy = round(accuracy_score(y_true, y_pred) * 100, 2)
            precision = round(precision_score(y_true, y_pred, average="weighted", zero_division=0) * 100, 2)
            recall = round(recall_score(y_true, y_pred, average="weighted", zero_division=0) * 100, 2)
            f1 = round(f1_score(y_true, y_pred, average="weighted", zero_division=0) * 100, 2)

            row = [model_name, accuracy, precision, recall, f1]

            color_code = COLORS.get(color, "")
            if color_code:
                row = [f"{color_code}{val}{RESET}" for val in row]

            summary_rows.append(row)

            # Per-category breakdown
            report = classification_report(y_true, y_pred, digits=2, output_dict=True, zero_division=0)
            classes = [key for key in report.keys() if key not in ["accuracy", "macro avg", "weighted avg"]]

            category_rows = []
            for cls in sorted(classes):  # sort for consistent order
                cls_metrics = report[cls]
                description = description_dict.get(cls, "")
                cat_row = [
                    cls,
                    description,
                    round(cls_metrics["precision"] * 100, 2),
                    round(cls_metrics["recall"] * 100, 2),
                    round(cls_metrics["f1-score"] * 100, 2),
                    int(cls_metrics["support"])
                ]
                if color_code:
                    cat_row = [f"{color_code}{v}{RESET}" for v in cat_row]
                category_rows.append(cat_row)

            # Add overall metrics row at the end
            overall_row = [
                "",
                f"Overall accuracy: {accuracy}%",
                precision,
                recall,
                f1,
                "—"
            ]
            if color_code:
                overall_row = [f"{color_code}{v}{RESET}" for v in overall_row]
            category_rows.append(overall_row)

            per_model_breakdowns.append((model_name, category_rows, section_break))

        except Exception as e:
            print(f"Error processing {model_name} ({file_path}): {e}")

    # Print overall summary table first
    headers_summary = ["Model", "Accuracy (%)", "Precision (%)", "Recall (%)", "F1 Score (%)"]
    display(Markdown(f"📊 **{task} — Overall Performance Summary**"))
    print(tabulate(summary_rows, headers=headers_summary, tablefmt=tablefmt_summary))

    # Then print per-category breakdowns
    for model_name, category_rows, section_break in per_model_breakdowns:
        headers_category = ["Label", "Description", "Precision (%)", "Recall (%)", "F1 Score (%)", "Support"]
        display(Markdown(f"🗂️ **{task} — {model_name} Per-Category Metrics**"))
        print(tabulate(category_rows, headers=headers_category, tablefmt=tablefmt_category))

        if section_break:
            print("\n" + "=" * 80 + "\n")

# GPT FEW SHOTS - HUMANITARIAN

In [30]:
model_results = [
    ("Zeroshot Text Only  - 4o", "test_results/humanitarian/Test-17/gpt-4o-Zeroshot-Text-Only.json", "red", False),
    ("OneShot Text Only   - 4o", "test_results/humanitarian/Test-17/gpt-4o-OneShot-Text-Only.json", "green", False),
    ("FiveShot Text Only  - 4o", "test_results/humanitarian/Test-17/gpt-4o-5shot-Text-Only.json", "blue", False),
    ("Fiveshots Text Only - 4o Inconsistent", "test_results/humanitarian/Test-17/gpt-4o-5shot-Inconsistent-Text-Only.json", "yellow", True),
    
    ("Zeroshot Image Only - 4o", "test_results/humanitarian/Test-17/gpt-4o-Zeroshot-Image-Only.json", "red", False),
    ("OneShot Image Only  - 4o", "test_results/humanitarian/Test-17/gpt-4o-Oneshot-Image-Only.json", "green", False),
    ("FiveShot Image Only - 4o", "test_results/humanitarian/Test-17/gpt-4o-5shot-Image-Only.json", "blue", True),    
    
    
    ("Zeroshot Text Image - 4o", "test_results/humanitarian/Test-17/gpt-4o-Zeroshot-Text-Image.json", "red", False),
    ("OneShot Text Image  - 4o", "test_results/humanitarian/Test-17/gpt-4o-Oneshot-Text-Image.json", "green", False),
    ("FiveShot Text Image - 4o", "test_results/humanitarian/Test-17/gpt-4o-5shot-Text-Image.json", "blue", True),  
    
  
    ("Zeroshot Text Only  - 4o mini", "test_results/humanitarian/Test-17/gpt-4o-mini-Zeroshot-Text-Only.json", "red", False),
    ("OneShot Text Only   - 4o mini", "test_results/humanitarian/Test-17/gpt-4o-mini-Oneshot-Text-Only.json", "green", False),
    ("FiveShot Text Only  - 4o mini", "test_results/humanitarian/Test-17/gpt-4o-mini-5shot-Text-Only.json", "blue", True),
    
    ("Zeroshot Image Only - 4o mini", "test_results/humanitarian/Test-17/gpt-4o-mini-Zeroshot-Image-Only.json", "red", False),
    ("OneShot Image Only  - 4o mini", "test_results/humanitarian/Test-17/gpt-4o-mini-Oneshot-Image-Only.json", "green", False),
    ("FiveShot Image Only - 4o mini", "test_results/humanitarian/Test-17/gpt-4o-mini-5shot-Image-Only.json", "blue", True),    
    
    
    ("Zeroshot Text Image - 4o mini", "test_results/humanitarian/Test-17/gpt-4o-mini-Zeroshot-Text-Image.json", "red", False),
    ("OneShot Text Image  - 4o mini", "test_results/humanitarian/Test-17/gpt-4o-mini-Oneshot-Text-Image.json", "green", False),
    ("FiveShot Text Image - 4o mini", "test_results/humanitarian/Test-17/gpt-4o-mini-5shot-Text-Image.json", "blue", False)    


]

#calculate_metrics_for_models(model_results, task="Tweet Humanitarian with GPT Fewshot")

calculate_metrics_with_category_breakdown(model_results, task="GPT Zeroshot & Fewshot - Humanitarian")


📊 **GPT Zeroshot & Fewshot - Humanitarian — Overall Performance Summary**

+---------------------------------------+----------------+-----------------+--------------+----------------+
| Model                                 |   Accuracy (%) |   Precision (%) |   Recall (%) |   F1 Score (%) |
+=======================================+================+=================+==============+================+
| Zeroshot Text Only  - 4o              |          74.66 |           75.03 |        74.66 |          74.27 |
+---------------------------------------+----------------+-----------------+--------------+----------------+
| OneShot Text Only   - 4o              |          71.1  |           76.59 |        71.1  |          72    |
+---------------------------------------+----------------+-----------------+--------------+----------------+
| FiveShot Text Only  - 4o              |          71.1  |           76.78 |        71.1  |          72.17 |
+---------------------------------------+----------------+-----------------+--------------+----------------+
| Fiveshots Text On

🗂️ **GPT Zeroshot & Fewshot - Humanitarian — Zeroshot Text Only  - 4o Per-Category Metrics**

+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| Label   | Description                     |   Precision (%) |   Recall (%) |   F1 Score (%) | Support   |
+=========+=================================+=================+==============+================+===========+
| 0       | Affected individuals            |           15.79 |        33.33 |          21.43 | 9         |
+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| 1       | Rescue/volunteering/donation    |           80.8  |        80.16 |          80.48 | 126       |
+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| 2       | Infrastructure & utility damage |           77.05 |        58.02 |          66.2  | 81        |
+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| 3       | Other relevant i

🗂️ **GPT Zeroshot & Fewshot - Humanitarian — OneShot Text Only   - 4o Per-Category Metrics**

...

+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| Label   | Description                     |   Precision (%) |   Recall (%) |   F1 Score (%) | Support   |
+=========+=================================+=================+==============+================+===========+
| 0       | Affected individuals            |           15.09 |        88.89 |          25.81 | 9         |
+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| 1       | Rescue/volunteering/donation    |           61.11 |        96.03 |          74.69 | 126       |
+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| 2       | Infrastructure & utility damage |           78.57 |        81.48 |          80    | 81        |
+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| 3       | Other relevant i

🗂️ **GPT Zeroshot & Fewshot - Humanitarian — OneShot Text Image  - 4o mini Per-Category Metrics**

+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| Label   | Description                     |   Precision (%) |   Recall (%) |   F1 Score (%) | Support   |
+=========+=================================+=================+==============+================+===========+
| 0       | Affected individuals            |           14.29 |        88.89 |          24.62 | 9         |
+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| 1       | Rescue/volunteering/donation    |           65.03 |        94.44 |          77.02 | 126       |
+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| 2       | Infrastructure & utility damage |           79.07 |        83.95 |          81.44 | 81        |
+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| 3       | Other relevant i

🗂️ **GPT Zeroshot & Fewshot - Humanitarian — FiveShot Text Image - 4o mini Per-Category Metrics**

+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| Label   | Description                     |   Precision (%) |   Recall (%) |   F1 Score (%) | Support   |
+=========+=================================+=================+==============+================+===========+
| 0       | Affected individuals            |           22.22 |        88.89 |          35.56 | 9         |
+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| 1       | Rescue/volunteering/donation    |           64.71 |        96.03 |          77.32 | 126       |
+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| 2       | Infrastructure & utility damage |           68.87 |        90.12 |          78.07 | 81        |
+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| 3       | Other relevant i

# LLAMA FEW-SHOTS - HUMANITARIAN

In [31]:
model_results = [
    ("Zeroshot Text Only  - Llama", "test_results/humanitarian/Test-23/Llama-3.2-11B-Huma-Zeroshot-Text-Only.json", "red", False),
    ("Oneshot Text Only  - Llama", "test_results/humanitarian/Test-23/Llama-3.2-11B-Huma-Oneshot-Text-Only.json", "green", False),
    ("Fiveshot Text Only  - Llama", "test_results/humanitarian/Test-23/Llama-3.2-11B-Huma-Fiveshot-Text-Only.json", "blue", True),
   
    ("Zeroshot Image Only  - Llama", "test_results/humanitarian/Test-23/Llama-3.2-11B-Huma-Zeroshot-Image-Only.json", "red", False),
    ("Oneshot Image Only   - Llama", "test_results/humanitarian/Test-23/Llama-3.2-11B-Huma-Oneshot-Image-Only.json", "green", True),

    ("Zeroshot Text Image  - Llama", "test_results/humanitarian/Test-23/Llama-3.2-11B-Huma-Zeroshot-Text-Image.json", "red", False),
    ("Oneshot Text Image   - Llama", "test_results/humanitarian/Test-23/Llama-3.2-11B-Huma-Oneshot-Text-Image.json", "green", False)   
]

calculate_metrics_with_category_breakdown(model_results, task="LLama Zeroshot & Fewshot - Humanitarian")


📊 **LLama Zeroshot & Fewshot - Humanitarian — Overall Performance Summary**

+------------------------------+----------------+-----------------+--------------+----------------+
| Model                        |   Accuracy (%) |   Precision (%) |   Recall (%) |   F1 Score (%) |
+==============================+================+=================+==============+================+
| Zeroshot Text Only  - Llama  |          74.45 |           78.43 |        74.45 |          74.09 |
+------------------------------+----------------+-----------------+--------------+----------------+
| Oneshot Text Only  - Llama   |          74.45 |           77.21 |        74.45 |          75.09 |
+------------------------------+----------------+-----------------+--------------+----------------+
| Fiveshot Text Only  - Llama  |          71.52 |           75.32 |        71.52 |          72.65 |
+------------------------------+----------------+-----------------+--------------+----------------+
| Zeroshot Image Only  - Llama |          76.54 |           76.79 |        76.54 |          75.66 |


🗂️ **LLama Zeroshot & Fewshot - Humanitarian — Zeroshot Text Only  - Llama Per-Category Metrics**

+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| Label   | Description                     |   Precision (%) |   Recall (%) |   F1 Score (%) | Support   |
+=========+=================================+=================+==============+================+===========+
| 0       | Affected individuals            |           35.29 |        66.67 |          46.15 | 9         |
+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| 1       | Rescue/volunteering/donation    |           76.39 |        87.3  |          81.48 | 126       |
+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| 2       | Infrastructure & utility damage |          100    |        27.16 |          42.72 | 81        |
+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| 3       | Other relevant i

🗂️ **LLama Zeroshot & Fewshot - Humanitarian — Oneshot Text Only  - Llama Per-Category Metrics**

...

+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| Label   | Description                     |   Precision (%) |   Recall (%) |   F1 Score (%) | Support   |
+=========+=================================+=================+==============+================+===========+
| 0       | Affected individuals            |           60    |        33.33 |          42.86 | 9         |
+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| 1       | Rescue/volunteering/donation    |           70.37 |        45.24 |          55.07 | 126       |
+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| 2       | Infrastructure & utility damage |           67.14 |        58.02 |          62.25 | 81        |
+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| 3       | Other relevant i

🗂️ **LLama Zeroshot & Fewshot - Humanitarian — Zeroshot Text Image  - Llama Per-Category Metrics**

+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| Label   | Description                     |   Precision (%) |   Recall (%) |   F1 Score (%) | Support   |
+=========+=================================+=================+==============+================+===========+
| 0       | Affected individuals            |           25    |        33.33 |          28.57 | 9         |
+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| 1       | Rescue/volunteering/donation    |           70    |        72.22 |          71.09 | 126       |
+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| 2       | Infrastructure & utility damage |          100    |        32.1  |          48.6  | 81        |
+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| 3       | Other relevant i

🗂️ **LLama Zeroshot & Fewshot - Humanitarian — Oneshot Text Image   - Llama Per-Category Metrics**

+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| Label   | Description                     |   Precision (%) |   Recall (%) |   F1 Score (%) | Support   |
+=========+=================================+=================+==============+================+===========+
| 0       | Affected individuals            |          100    |        11.11 |          20    | 9         |
+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| 1       | Rescue/volunteering/donation    |           72.81 |        65.87 |          69.17 | 126       |
+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| 2       | Infrastructure & utility damage |           64    |        59.26 |          61.54 | 81        |
+---------+---------------------------------+-----------------+--------------+----------------+-----------+
| 3       | Other relevant i

# LLAMA FEW-SHOTS - INFORMATIVE

In [32]:
model_results = [
    ("Zeroshot Text Only  - Llama", "test_results/informative/Test-23/Llama-3.2-11B-Info-Zeroshot-Text-Only.json", "red", False),
    ("Oneshot Text Only  - Llama", "test_results/informative/Test-23/Llama-3.2-11B-Info-Oneshot-Text-Only.json", "green", False),
    ("Fiveshot Text Only  - Llama", "test_results/informative/Test-23/Llama-3.2-11B-Info-Fiveshot-Text-Only.json", "blue", True),
   
    ("Zeroshot Image Only  - Llama", "test_results/informative/Test-23/Llama-3.2-11B-Info-Zeroshot-Image-Only.json", "red", False),
    ("Oneshot Image Only   - Llama", "test_results/informative/Test-23/Llama-3.2-11B-Info-Oneshot-Image-Only.json", "green", True),

    ("Zeroshot Text Image  - Llama", "test_results/informative/Test-23/Llama-3.2-11B-Info-Zeroshot-Text-Image.json", "red", False),
    ("Oneshot Text Image   - Llama", "test_results/informative/Test-23/Llama-3.2-11B-Info-Oneshot-Text-Image.json", "green", False)
]

calculate_metrics_with_category_breakdown(model_results, task="LLama Zeroshot & Fewshot - Informative")

📊 **LLama Zeroshot & Fewshot - Informative — Overall Performance Summary**

+------------------------------+----------------+-----------------+--------------+----------------+
| Model                        |   Accuracy (%) |   Precision (%) |   Recall (%) |   F1 Score (%) |
+==============================+================+=================+==============+================+
| Zeroshot Text Only  - Llama  |          83.57 |           83.28 |        83.57 |          83.22 |
+------------------------------+----------------+-----------------+--------------+----------------+
| Oneshot Text Only  - Llama   |          82.79 |           84.46 |        82.79 |          81.22 |
+------------------------------+----------------+-----------------+--------------+----------------+
| Fiveshot Text Only  - Llama  |          82.53 |           83.82 |        82.53 |          81.04 |
+------------------------------+----------------+-----------------+--------------+----------------+
| Zeroshot Image Only  - Llama |          86.05 |           86.57 |        86.05 |          85.34 |


🗂️ **LLama Zeroshot & Fewshot - Informative — Zeroshot Text Only  - Llama Per-Category Metrics**

+---------+--------------------------+-----------------+--------------+----------------+-----------+
| Label   | Description              |   Precision (%) |   Recall (%) |   F1 Score (%) | Support   |
+=========+==========================+=================+==============+================+===========+
| 0       | Not informative          |           79.17 |        67.86 |          73.08 | 504       |
+---------+--------------------------+-----------------+--------------+----------------+-----------+
| 1       | Informative              |           85.3  |        91.26 |          88.18 | 1030      |
+---------+--------------------------+-----------------+--------------+----------------+-----------+
|         | Overall accuracy: 83.57% |           83.28 |        83.57 |          83.22 | —         |
+---------+--------------------------+-----------------+--------------+----------------+-----------+


🗂️ **LLama Zeroshot & Fewshot - Informative — Oneshot Text Only  - Llama Per-Category Metrics**

...

+---------+--------------------------+-----------------+--------------+----------------+-----------+
| Label   | Description              |   Precision (%) |   Recall (%) |   F1 Score (%) | Support   |
+=========+==========================+=================+==============+================+===========+
| 0       | Not informative          |           95.61 |        21.63 |          35.28 | 504       |
+---------+--------------------------+-----------------+--------------+----------------+-----------+
| 1       | Informative              |           72.18 |        99.51 |          83.67 | 1030      |
+---------+--------------------------+-----------------+--------------+----------------+-----------+
|         | Overall accuracy: 73.92% |           79.88 |        73.92 |          67.77 | —         |
+---------+--------------------------+-----------------+--------------+----------------+-----------+




🗂️ **LLama Zeroshot & Fewshot - Informative — Zeroshot Text Image  - Llama Per-Category Metrics**

+---------+--------------------------+-----------------+--------------+----------------+-----------+
| Label   | Description              |   Precision (%) |   Recall (%) |   F1 Score (%) | Support   |
+=========+==========================+=================+==============+================+===========+
| 0       | Not informative          |           97.58 |        40.08 |          56.82 | 504       |
+---------+--------------------------+-----------------+--------------+----------------+-----------+
| 1       | Informative              |           77.24 |        99.51 |          86.97 | 1030      |
+---------+--------------------------+-----------------+--------------+----------------+-----------+
|         | Overall accuracy: 79.99% |           83.93 |        79.99 |          77.07 | —         |
+---------+--------------------------+-----------------+--------------+----------------+-----------+


🗂️ **LLama Zeroshot & Fewshot - Informative — Oneshot Text Image   - Llama Per-Category Metrics**

+---------+--------------------------+-----------------+--------------+----------------+-----------+
| Label   | Description              |   Precision (%) |   Recall (%) |   F1 Score (%) | Support   |
+=========+==========================+=================+==============+================+===========+
| 0       | Not informative          |           49.14 |        79.37 |          60.7  | 504       |
+---------+--------------------------+-----------------+--------------+----------------+-----------+
| 1       | Informative              |           85.56 |        59.81 |          70.4  | 1030      |
+---------+--------------------------+-----------------+--------------+----------------+-----------+
|         | Overall accuracy: 66.23% |           73.59 |        66.23 |          67.21 | —         |
+---------+--------------------------+-----------------+--------------+----------------+-----------+


# GPT FEW-SHOTS - INFORMATIVE

In [33]:
model_results = [
    ("Zeroshot Text Only  - 4o", "test_results/informative/Test-23/gpt-4o-Info-Zeroshot-Text-Only.json", "red", False),
    ("OneShot Text Only   - 4o", "test_results/informative/Test-23/gpt-4o-Info-OneShot-Text-Only.json", "green", True),

    ("Zeroshot Image Only - 4o", "test_results/informative/Test-23/gpt-4o-Info-Zeroshot-Image-Only.json", "red", False),
    ("OneShot Image Only  - 4o", "test_results/informative/Test-23/gpt-4o-Info-Oneshot-Image-Only.json", "green", True),       
    
    ("Zeroshot Text Image - 4o", "test_results/informative/Test-23/gpt-4o-Info-Zeroshot-Text-Image.json", "red", False),
    ("OneShot Text Image  - 4o", "test_results/informative/Test-23/gpt-4o-Info-Oneshot-Text-Image.json", "green", True),
      
    ("Zeroshot Text Only  - 4o mini", "test_results/informative/Test-23/gpt-4o-mini-Info-Zeroshot-Text-Only.json", "red", False),
    ("OneShot Text Only   - 4o mini", "test_results/informative/Test-23/gpt-4o-mini-Info-Oneshot-Text-Only.json", "green", True),
    
    ("Zeroshot Image Only - 4o mini", "test_results/informative/Test-23/gpt-4o-mini-Info-Zeroshot-Image-Only.json", "red", False),
    ("OneShot Image Only  - 4o mini", "test_results/informative/Test-23/gpt-4o-mini-Info-Oneshot-Image-Only.json", "green", True),
    
    ("Zeroshot Text Image - 4o mini", "test_results/informative/Test-23/gpt-4o-mini-Info-Zeroshot-Text-Image.json", "red", False),
    ("OneShot Text Image  - 4o mini", "test_results/informative/Test-23/gpt-4o-mini-Info-Oneshot-Text-Image.json", "green", False),   
]

calculate_metrics_with_category_breakdown(model_results, task="GPT Zeroshot & Fewshot - Informative")

📊 **GPT Zeroshot & Fewshot - Informative — Overall Performance Summary**

+-------------------------------+----------------+-----------------+--------------+----------------+
| Model                         |   Accuracy (%) |   Precision (%) |   Recall (%) |   F1 Score (%) |
+===============================+================+=================+==============+================+
| Zeroshot Text Only  - 4o      |          79.47 |           81.38 |        79.47 |          79.93 |
+-------------------------------+----------------+-----------------+--------------+----------------+
| OneShot Text Only   - 4o      |          84.16 |           83.94 |        84.16 |          83.99 |
+-------------------------------+----------------+-----------------+--------------+----------------+
| Zeroshot Image Only - 4o      |          85.14 |           85.58 |        85.14 |          85.29 |
+-------------------------------+----------------+-----------------+--------------+----------------+
| OneShot Image Only  - 4o      |          86.38 |           86.38 |        86.38 |        

🗂️ **GPT Zeroshot & Fewshot - Informative — Zeroshot Text Only  - 4o Per-Category Metrics**

+---------+--------------------------+-----------------+--------------+----------------+-----------+
| Label   | Description              |   Precision (%) |   Recall (%) |   F1 Score (%) | Support   |
+=========+==========================+=================+==============+================+===========+
| 0       | Not informative          |           65.12 |        80.75 |          72.1  | 504       |
+---------+--------------------------+-----------------+--------------+----------------+-----------+
| 1       | Informative              |           89.33 |        78.83 |          83.75 | 1030      |
+---------+--------------------------+-----------------+--------------+----------------+-----------+
|         | Overall accuracy: 79.47% |           81.38 |        79.47 |          79.93 | —         |
+---------+--------------------------+-----------------+--------------+----------------+-----------+


🗂️ **GPT Zeroshot & Fewshot - Informative — OneShot Text Only   - 4o Per-Category Metrics**

...

+---------+--------------------------+-----------------+--------------+----------------+-----------+
| Label   | Description              |   Precision (%) |   Recall (%) |   F1 Score (%) | Support   |
+=========+==========================+=================+==============+================+===========+
| 0       | Not informative          |           90.86 |        73.02 |          80.97 | 504       |
+---------+--------------------------+-----------------+--------------+----------------+-----------+
| 1       | Informative              |           87.95 |        96.41 |          91.99 | 1030      |
+---------+--------------------------+-----------------+--------------+----------------+-----------+
|         | Overall accuracy: 88.72% |           88.91 |        88.72 |          88.37 | —         |
+---------+--------------------------+-----------------+--------------+----------------+-----------+




🗂️ **GPT Zeroshot & Fewshot - Informative — Zeroshot Text Image - 4o mini Per-Category Metrics**

+---------+--------------------------+-----------------+--------------+----------------+-----------+
| Label   | Description              |   Precision (%) |   Recall (%) |   F1 Score (%) | Support   |
+=========+==========================+=================+==============+================+===========+
| 0       | Not informative          |           96.1  |        58.73 |          72.91 | 504       |
+---------+--------------------------+-----------------+--------------+----------------+-----------+
| 1       | Informative              |           83.03 |        98.83 |          90.25 | 1030      |
+---------+--------------------------+-----------------+--------------+----------------+-----------+
|         | Overall accuracy: 85.66% |           87.33 |        85.66 |          84.55 | —         |
+---------+--------------------------+-----------------+--------------+----------------+-----------+


🗂️ **GPT Zeroshot & Fewshot - Informative — OneShot Text Image  - 4o mini Per-Category Metrics**

+---------+--------------------------+-----------------+--------------+----------------+-----------+
| Label   | Description              |   Precision (%) |   Recall (%) |   F1 Score (%) | Support   |
+=========+==========================+=================+==============+================+===========+
| 0       | Not informative          |           93.91 |        67.26 |          78.38 | 504       |
+---------+--------------------------+-----------------+--------------+----------------+-----------+
| 1       | Informative              |           85.93 |        97.86 |          91.51 | 1030      |
+---------+--------------------------+-----------------+--------------+----------------+-----------+
|         | Overall accuracy: 87.81% |           88.55 |        87.81 |          87.2  | —         |
+---------+--------------------------+-----------------+--------------+----------------+-----------+
